In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Sjögrens_Syndrome/GSE140161'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Systems biology demonstrates the predominant role of circulating interferon-alpha in primary Sjögren's syndrome and a genetic association with the class II HLA DQ locus"
!Series_summary	"Primary Sjögren’s syndrome (pSS) is the second most frequent systemic autoimmune disease, affecting 0.1% of the general population. No specific immunomodulatory drug has demonstrated efficacy for this disease, and no biomarker is available to identify patients at risk of developing systemic complications. To characterize the molecular and clinical variability across pSS patients, we integrated transcriptomic, proteomic, cellular and genetic data with clinical phenotypes in a cohort of 351 pSS patients. Unbiased global transcriptomic analysis revealed an IFN gene signature as the strongest driver of transcriptomic variability. The resulting stratification was replicated in three independent cohorts. As transcriptomic analysis did not discriminate between type I and

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on series overall design 'Whole blood transcriptome'

# Identify the keys for each variable
# 'Sjögren’s_Syndrome':
if len(set(['Sjögren’s syndrome'])) > 1:
    trait_row = 4
else:
    trait_row = None

# 'age': Not explicitly available in the sample characteristics dictionary.
age_row = None

# 'gender':
if len(set(['female', 'male'])) > 1:
    gender_row = 1
else:
    gender_row = None

# Functions for data type conversion
def convert_trait(value):
    return 1  # Since 'Sjögren’s syndrome' is the only value, we'll always return 1

def convert_age(value):
    return None  # No age data available

def convert_gender(value):
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE140161', './preprocessed/Sjögrens_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Sjögrens_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Sjögrens_Syndrome/trait_data/GSE140161.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
